In [22]:
def read_sent(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[1])
    return(ents)

def read_labels(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[2])
    return(ents)

def read_index(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[0])
    return(ents)

In [49]:
#Training data

#returns list of lists
training_labels = read_labels("baseline_data/en_ewt-ud-train.iob2")
training_sent = read_sent("baseline_data/en_ewt-ud-train.iob2")

#flatten to one list to be able to use myutils
train_labels = sum(training_labels, [])
train_sent = sum(training_sent, [])

In [53]:
#Evaluation data

dev_labels = read_labels("baseline_data/en_ewt-ud-dev.iob2")
dev_sent = read_sent("baseline_data/en_ewt-ud-dev.iob2")

dev_flat_labels = sum(dev_labels, [])
dev_flat_sent = sum(dev_sent, [])

In [25]:
#Test data
#Keeping track of indeces to save to required .iob2 format for model's predictions

test_labels = read_labels("baseline_data/en_ewt-ud-test.iob2")
test_sent = read_sent("baseline_data/en_ewt-ud-test.iob2")
test_index = read_index("baseline_data/en_ewt-ud-test.iob2")

test_labels = sum(test_labels, [])
test_sent = sum(test_sent, [])
test_index = sum(test_index, [])

In [26]:
label_names = set(test_labels)

In [27]:
label_names

{'B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O'}

In [28]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [87]:
inputs = []
for sentence in training_sent:
    inputt = tokenizer(sentence, is_split_into_words=True)
    inputs.append(inputt)

In [125]:
inputs

[{'input_ids': [101, 2777, 1107, 1103, 1362, 1110, 146, 13855, 10337, 136, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 146, 13855, 10337, 6230, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 15268, 1193, 1737, 1106, 1129, 1141, 1104, 1103, 1211, 14449, 16962, 1116, 1107, 1103, 1362, 117, 1103, 146, 13855, 10337, 6230, 1113, 1103, 3070, 1104, 4904, 1105, 3524, 117, 1132, 170, 4664, 1538, 1267, 8322, 1107, 1103, 1298, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 1109, 2642, 1104, 1103, 4887, 1110, 1227, 6889, 1112, 786, 144, 1813, 24581, 1161, 9352, 12120, 6639, 2858, 787, 1

In [92]:
inputs[0].word_ids()

[None, 0, 1, 2, 3, 4, 5, 5, 5, 6, None]

In [38]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [135]:
import myutils
UNK = "[UNK]"

id2label, label2id = myutils.labels2lookup(train_labels, UNK)
NLABELS = len(id2label)
print(train_labels)
print(label2id)

#converting BIO labels to numerical labels
train_labels_num = [label2id.get(label, label2id[UNK]) for label in train_labels]

[1, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 2, 3, 3, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [103]:
labels = training_labels
word_ids = []
for i in inputs:
    word_ids.append(i.word_ids())
word_ids_flat = sum(word_ids, [])
aligned_training = align_labels_with_tokens(train_labels_num, word_ids_flat)


[['O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], ['B-LOC', 'I-LOC'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O'], ['O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 

In [122]:
def list_to_sentences(lst):
    sentences = []
    current_sentence = []
    
    for item in lst:
        if item == -100:
            if current_sentence:
                sentences.append(current_sentence)
                current_sentence = []
        else:
            current_sentence.append(item)
    
    if current_sentence:
        sentences.append(current_sentence)
    
    return sentences


sentences = list_to_sentences(aligned_training)
print(sentences)


[[1, 1, 1, 1, 1, 2, 2, 2, 1], [1, 2, 2, 1], [1, 2, 1, 1, 1, 1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 1, 1], [1, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1], [1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 4, 4, 1, 1, 1], [1, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 3, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1], [1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1], [1, 1, 1, 1, 1, 2, 1, 2, 3, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 3, 1, 1, 1, 1, 2], [1, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 2, 2, 2, 1, 2], [1, 1, 1, 1, 1, 2, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1], [1, 2, 1, 1, 1, 1, 2, 1, 2, 3, 1, 1, 1

In [124]:
to_zip_train = [[-100] + sublist + [-100] for sublist in sentences]

In [128]:
# Assuming inputs and to_zip_train are defined as provided

# Zip inputs with to_zip_train and add 'label' key to each item in inputs
for input_item, label_item in zip(inputs, to_zip_train):
    input_item['label'] = label_item

# Print the updated inputs list
print(inputs[0])


{'input_ids': [101, 2777, 1107, 1103, 1362, 1110, 146, 13855, 10337, 136, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': [-100, 1, 1, 1, 1, 1, 2, 2, 2, 1, -100]}


In [163]:
inputs_dev = []
for sentence in dev_sent:
    inputt = tokenizer(sentence, is_split_into_words=True)
    inputs_dev.append(inputt)

In [165]:
dev_ner_labels = [label2id.get(label, label2id[UNK]) for label in dev_flat_labels]

In [166]:
word_ids = []
for i in inputs_dev:
    word_ids.append(i.word_ids())
word_ids_flat = sum(word_ids, [])
aligned_dev = align_labels_with_tokens(dev_ner_labels, word_ids_flat)

In [167]:
to_zip_dev = [[-100] + sublist + [-100] for sublist in list_to_sentences(aligned_dev)]

In [170]:
for input_item, label_item in zip(inputs_dev, to_zip_dev):
    input_item['label'] = label_item

# Print the updated inputs list
print(inputs_dev[0])

{'input_ids': [101, 1187, 1169, 146, 1243, 182, 1766, 23694, 1116, 1107, 27629, 8223, 1161, 5952, 117, 146, 1209, 1176, 1103, 170, 14527, 17054, 2076, 117, 1133, 146, 1209, 1106, 2222, 1330, 1116, 4268, 136, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]}


In [105]:
inputs_dev = []
for sentence in dev_sent:
    inputt = tokenizer(sentence, is_split_into_words=True)
    inputs.append(inputt)

In [106]:
dev_ner_labels = [label2id.get(label, label2id[UNK]) for label in dev_flat_labels]


In [107]:
word_ids = []
for i in inputs:
    word_ids.append(i.word_ids())
word_ids_flat = sum(word_ids, [])
aligned_dev = align_labels_with_tokens(dev_ner_labels, word_ids_flat)

In [55]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [130]:
batch = data_collator([inputs[i] for i in range(2)])
batch["label"]

tensor([[-100,    1,    1,    1,    1,    1,    2,    2,    2,    1, -100],
        [-100,    1,    2,    2,    1, -100, -100, -100, -100, -100, -100]])

In [132]:
import evaluate

metric = evaluate.load("seqeval")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [133]:
id2label = {v: k for k, v in label2id.items()}

In [145]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label = id2label
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['cl

In [143]:
model.config.num_labels

8

In [146]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [171]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [172]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=inputs,
    eval_dataset=inputs_dev,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

LocalTokenNotFoundError: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.